# dbt learn on demand example

In [2]:
#@title Mount your Google Drive to save and use local files
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

% cd "/content/gdrive/My Drive/Colab Notebooks/dbt"

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/dbt


In [3]:
#@title Install Tinybird CLI and your token
!pip install tinybird-cli -q -U

import os

if not os.path.isfile('.tinyb'):
  !tb auth

!tb init

     |████████████████████████████████| 71 kB 4.4 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 61 kB 6.5 MB/s 
     |████████████████████████████████| 46 kB 3.5 MB/s 
     |████████████████████████████████| 81 kB 4.7 MB/s 
     |████████████████████████████████| 86 kB 3.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.25.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
** - /datasources already exists, skipping
** - /datasources/fixtures already exists, skipping
** - /endpoints already exists, skipping
** - /explorations already exists, skipping
** - /pipes already exists, skipping


In [4]:
#@title Helper function
def write_text_to_file(filename, text):
  with open(filename, 'w') as f: f.write(text)

# Training Data

In [48]:
!tb datasource generate 'http://dbt-tutorial-public.s3-us-west-2.amazonaws.com/jaffle_shop_orders.csv'

** Generated datasources/jaffle_shop_orders.datasource
** => Create it on the server running: $ tb push datasources/jaffle_shop_orders.datasource
** => Append data using: $ tb datasource append jaffle_shop_orders http://dbt-tutorial-public.s3-us-west-2.amazonaws.com/jaffle_shop_orders.csv`



In [49]:
!cat datasources/jaffle_shop_orders.datasource

DESCRIPTION generated from http://dbt-tutorial-public.s3-us-west-2.amazonaws.com/jaffle_shop_orders.csv

SCHEMA >
    `_id` Int16,
    `user_id` Int16,
    `order_date` Date,
    `status` String

In [50]:
!tb push datasources/jaffle_shop_orders.datasource
!tb datasource append jaffle_shop_orders http://dbt-tutorial-public.s3-us-west-2.amazonaws.com/jaffle_shop_orders.csv

** Processing datasources/jaffle_shop_orders.datasource
** Building dependencies
** Running jaffle_shop_orders 
** 'jaffle_shop_orders' created
** Not pushing fixtures
** 🥚 starting import process

** There was an error with file contents: 1 row in quarantine and 1 invalid line.


In [51]:
!tb datasource generate 'http://dbt-tutorial-public.s3-us-west-2.amazonaws.com/jaffle_shop_customers.csv'

** Generated datasources/jaffle_shop_customers.datasource
** => Create it on the server running: $ tb push datasources/jaffle_shop_customers.datasource
** => Append data using: $ tb datasource append jaffle_shop_customers http://dbt-tutorial-public.s3-us-west-2.amazonaws.com/jaffle_shop_customers.csv`



In [52]:
!cat datasources/jaffle_shop_customers.datasource

DESCRIPTION generated from http://dbt-tutorial-public.s3-us-west-2.amazonaws.com/jaffle_shop_customers.csv

SCHEMA >
    `_id` Int16,
    `first_name` String,
    `last_name` String

In [53]:
!tb push datasources/jaffle_shop_customers.datasource
!tb datasource append jaffle_shop_customers http://dbt-tutorial-public.s3-us-west-2.amazonaws.com/jaffle_shop_customers.csv

** Processing datasources/jaffle_shop_customers.datasource
** Building dependencies
** Running jaffle_shop_customers 
** 'jaffle_shop_customers' created
** Not pushing fixtures
** 🥚 starting import process

** There was an error with file contents: 1 row in quarantine and 1 invalid line.


In [54]:
!tb datasource generate 'http://dbt-tutorial-public.s3-us-west-2.amazonaws.com/stripe_payments.csv'

** Generated datasources/stripe_payments.datasource
** => Create it on the server running: $ tb push datasources/stripe_payments.datasource
** => Append data using: $ tb datasource append stripe_payments http://dbt-tutorial-public.s3-us-west-2.amazonaws.com/stripe_payments.csv`



In [55]:
!cat datasources/stripe_payments.datasource

DESCRIPTION generated from http://dbt-tutorial-public.s3-us-west-2.amazonaws.com/stripe_payments.csv

SCHEMA >
    `_id` Int16,
    `orderid` Int16,
    `paymentmethod` String,
    `status` String,
    `amount` Int32,
    `created` Date

In [56]:
!tb push datasources/stripe_payments.datasource
!tb datasource append stripe_payments http://dbt-tutorial-public.s3-us-west-2.amazonaws.com/stripe_payments.csv

** Processing datasources/stripe_payments.datasource
** Building dependencies
** Running stripe_payments 
** 'stripe_payments' created
** Not pushing fixtures
** 🥚 starting import process

** There was an error with file contents: 1 row in quarantine and 1 invalid line.


# Staging Pipes with Tests

### Customers

In [64]:
filename = 'pipes/stg_customers.pipe'
text = '''
DESCRIPTION staging_customers
NODE staging_customers
SQL >
  SELECT
    _id as customer_id,
    first_name,
    last_name 

  FROM jaffle_shop_customers
'''

write_text_to_file(filename, text)

In [65]:
!tb push 'pipes/stg_customers.pipe'

** Processing pipes/stg_customers.pipe
** Building dependencies
** Running stg_customers 
** => Test endpoint at https://api.tinybird.co/v0/pipes/stg_customers.json
** 'stg_customers' created
** Not pushing fixtures


#### Customer Tests

#### Not Null

Dowloading the schema for `jaffle_shop_customers`:
```
SCHEMA >
    `_id` Int16,
    `first_name` String,
    `last_name` String

ENGINE "MergeTree"
ENGINE_PARTITION_KEY "substring(toString(_id), 1, 1)"
ENGINE_SORTING_KEY "_id, cityHash64(last_name)"
ENGINE_SAMPLING_KEY "cityHash64(last_name)"
````
Reveals that '_id' is part of the ENGINE_SORTING_KEY, which by definition is NOT NULL.

#### Unique

In [74]:
filename = 'pipes/chk_unique_customers.pipe'
text = '''
DESCRIPTION checking uniqueness of _id
NODE check_unique_id
SQL >
  SELECT
    count(_id)-uniq(_id)
  FROM jaffle_shop_customers
'''

write_text_to_file(filename, text)

!tb push 'pipes/chk_unique_customers.pipe'

### Orders

In [91]:
filename = 'pipes/stg_orders.pipe'
text = '''
DESCRIPTION staging orders
NODE staging_orders
SQL >
    SELECT 
        _id as order_id,
        user_id as customer_id,
        order_date,
        status
    FROM jaffle_shop_orders
'''

write_text_to_file(filename, text)

!tb push 'pipes/stg_orders.pipe'

** Processing pipes/stg_orders.pipe
** Building dependencies
** Warning: stg_orders already exists, skipping
** Not pushing fixtures


#### Orders Tests

#### Not Null

Dowloading the schema for jaffle_shop_orders: 

```
SCHEMA >
    `_id` Int16,
    `user_id` Int16,
    `order_date` Date,
    `status` String

ENGINE "MergeTree"
ENGINE_PARTITION_KEY "toYear(order_date)"
ENGINE_SORTING_KEY "order_date, cityHash64(status)"
ENGINE_SAMPLING_KEY "cityHash64(status)"
```
Reveals that '_id' is part of the ENGINE_SORTING_KEY, which by definition is NOT NULL.

#### Unique

In [76]:
filename = 'pipes/chk_unique_orders.pipe'
text = '''
DESCRIPTION checking uniqueness of _id
NODE check_unique_id
SQL >
  SELECT
    count(_id)-uniq(_id)
  FROM jaffle_shop_orders
'''

write_text_to_file(filename, text)

!tb push 'pipes/chk_unique_orders.pipe'

** Processing pipes/chk_unique_orders.pipe
** Building dependencies
** Running chk_unique_orders 
** => Test endpoint at https://api.tinybird.co/v0/pipes/chk_unique_orders.json
** 'chk_unique_orders' created
** Not pushing fixtures


#### Accepted Values

In [77]:
filename = 'pipes/accepted_values_orders.pipe'
text = '''
DESCRIPTION accepted values orders
NODE check_accepted_values
SQL >
  SELECT
    DISTINCT status
  FROM jaffle_shop_orders
'''

write_text_to_file(filename, text)

!tb push 'pipes/accepted_values_orders.pipe'

** Processing pipes/accepted_values_orders.pipe
** Building dependencies
** Running accepted_values_orders 
** => Test endpoint at https://api.tinybird.co/v0/pipes/accepted_values_orders.json
** 'accepted_values_orders' created
** Not pushing fixtures


### Payments

In [78]:
filename = 'pipes/stg_payments.pipe'
text = '''
DESCRIPTION staging payments
NODE staging_payments
SQL >
  SELECT
    _id as payment_id,
    orderid as order_id,
    paymentmethod as payment_method,
    status,

    -- amount is stored in cents, convert it to dollars
    amount / 100 as amount,
    created as created_at

  FROM stripe_payments
'''

write_text_to_file(filename, text)

!tb push 'pipes/stg_payments.pipe'

** Processing pipes/stg_payments.pipe
** Building dependencies
** Running stg_payments 
** => Test endpoint at https://api.tinybird.co/v0/pipes/stg_payments.json
** 'stg_payments' created
** Not pushing fixtures


# Fact Table for Orders

In [94]:
filename = 'pipes/fct_orders.pipe'
text = '''
DESCRIPTION >
	fact table orders

NODE order_payments
SQL >

    SELECT
      order_id,
      SUM(case when status = 'success' then amount end) AS amount
    FROM stg_payments
    GROUP BY order_id

NODE final_fct_orders
SQL >

    SELECT
      order_id,
      customer_id,
      order_date,
      amount
    FROM stg_orders
    LEFT JOIN order_payments
    USING order_id
'''

write_text_to_file(filename, text)

!tb push 'pipes/fct_orders.pipe'

** Processing pipes/fct_orders.pipe
** Building dependencies
** Running fct_orders 
** => Test endpoint at https://api.tinybird.co/v0/pipes/fct_orders.json
** 'fct_orders' created
** Not pushing fixtures


# Dimension Table for Customers

In [8]:
filename = 'pipes/dim_customers.pipe'
text = '''
DESCRIPTION dimensions table customers
NODE orders_by_customer
SQL >
    SELECT 
            customer_id,
            min(order_date) as first_order_date,
            max(order_date) as most_recent_order_date,
            count(order_id) as number_of_orders,
            sum(amount) as lifetime_value
     FROM fct_orders
     GROUP BY customer_id

NODE dim_customers_node
SQL >
    SELECT 
            customer_id,
            first_name,
            last_name,
            first_order_date,
            most_recent_order_date,
            number_of_orders,
            lifetime_value
    FROM stg_customers
    LEFT JOIN orders_by_customer 
    USING customer_id
    ORDER BY customer_id
'''

write_text_to_file(filename, text)

!tb push 'pipes/dim_customers.pipe'

** Processing pipes/dim_customers.pipe
** Building dependencies
** Running dim_customers 
** => Test endpoint at https://api.tinybird.co/v0/pipes/dim_customers.json
** 'dim_customers' created
** Not pushing fixtures
